# Beetaloo: MICE (max_iter=20)

In [1]:
# import libraries
import numpy as np
import pandas as pd

import json

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import missingno as msno

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import KFold, GroupKFold

from sklearn.model_selection import ParameterGrid

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import time


## 1. Data Pre-processing

In [2]:
# beetaloo dataset
beetaloo = pd.read_csv(filepath_or_buffer='dataset_beetaloo.csv', low_memory=False)
beetaloo.head()

,WELL_ID,YEAR,LATITUDE,LONGITUDE,DEPTH,DENSITY,GR,SONIC,RESISTIVITY,SP,NEUTRON,CALIPER,BITSIZE,TOC_CORE,TOC_CUTTINGS,LITHO,ORDER_3,ORDER_4
0,Alexander_1,1988.0,-15.16911,134.855921,63.1,NaN,111.753,NaN,NaN,NaN,NaN,4.705,NaN,NaN,NaN,Velkerri Formation,HST-2,RST-2.5
1,Alexander_1,1988.0,-15.16911,134.855921,63.2,NaN,108.300,NaN,NaN,NaN,NaN,4.701,NaN,NaN,NaN,Velkerri Formation,HST-2,RST-2.5
2,Alexander_1,1988.0,-15.16911,134.855921,63.3,NaN,105.699,NaN,NaN,NaN,NaN,4.676,NaN,NaN,NaN,Velkerri Formation,HST-2,RST-2.5
3,Alexander_1,1988.0,-15.16911,134.855921,63.4,NaN,106.119,NaN,NaN,NaN,NaN,4.668,NaN,NaN,NaN,Velkerri Formation,HST-2,RST-2.5
4,Alexander_1,1988.0,-15.16911,134.855921,63.5,NaN,106.734,NaN,NaN,NaN,NaN,4.691,NaN,NaN,NaN,Velkerri Formation,HST-2,RST-2.5


In [3]:
# rename columns
beetaloo.rename(columns={'DENSITY': 'RHOB', 
                         'SONIC': 'DT',
                         'RESISTIVITY': 'RES', 
                         'NEUTRON': 'NPHI',
                         'ORDER_4': 'STRAT'
                        }, inplace=True
               )

In [4]:
# complete the years of some wells
wells_no_year = ['Balmain_1', 'Jamison_1', 'Lady_Penrhyn_1', 'Ronald_1', 'Scarborough_1', 'Sever_1', 'Walton_2']
years = [1992, 1991, 1987, 1993, 1987, 1990, 1989]

for i in range(len(wells_no_year)):
    beetaloo.loc[beetaloo['WELL_ID'] == wells_no_year[i], 'YEAR'] = years[i] 
    
    
# complete the location of some wells
wells_no_location = ['Balmain_1', 'Birdum_Creek_1', 'Broadmere_1', 'Jamison_1', 'Lady_Penrhyn_1', 'Marmbulligan_1', 'Ronald_1', 'Scarborough_1', 'Sever_1', 'Tarlee_1', 'Walton_2', 'Wyworrie_1']

x_coord = [-92402.40724, 142017.0021, -24907.23409, -501.9841485, 82970.28312, 17486.61935, 86665.76837, -125242.803, -124956.7945, -38322.74503, -135702.1635]
y_coord = [-1809383.515, -1887528.408, -1936401.136, -1747313.691, -1872700.38, -1877801.953, -1759280.809, -1766817.469, -1845740.884, -1839970.706, -1780656.283]

lat = [-16.620444] 
lon = [133.577361]



# convert GDA94 to WGS84
import pyproj
# GDA94 / Geoscience Australia Lambert, EPSG:3112
# WGS 84 -- WGS84 - World Geodetic System 1984, used in GPS, EPSG:4326
gda94_wgs84 = pyproj.Transformer.from_crs(3112, 4326)
for i in range(len(x_coord)):
    new_coord = gda94_wgs84.transform(x_coord[i], y_coord[i])
    lat.append(new_coord[0])
    lon.append(new_coord[1])

# replace coordinates in the dataset
for i in range(len(wells_no_location)):
    beetaloo.loc[beetaloo['WELL_ID'] == wells_no_location[i], 'LATITUDE'] = lat[i]
    beetaloo.loc[beetaloo['WELL_ID'] == wells_no_location[i], 'LONGITUDE'] = lon[i]

In [5]:
beetaloo.rename(columns={'LONGITUDE': 'X',
                         'LATITUDE' : 'Y'}, inplace=True)

In [6]:
# drop columns
beetaloo.drop(['YEAR', 'CALIPER', 'BITSIZE', 'TOC_CORE', 'TOC_CUTTINGS', 'ORDER_3'], axis=1, inplace=True)

In [7]:
# replace sonic values smaller than 0 for NaN values
mask = beetaloo['DT'] < 0
beetaloo.loc[mask, 'DT'] = np.nan

# replace resistivity values smaller than or equal to 0 for NaN values
mask = beetaloo['RES'] <= 0
beetaloo.loc[mask, 'RES'] = np.nan

# create a new column to store log base 10 of resistivity
beetaloo['RES_10'] = np.log10(beetaloo['RES']+1)

# replace neutron porosity values smaller than 0 for NaN values
mask = beetaloo['NPHI'] < 0
beetaloo.loc[mask, 'NPHI'] = np.nan

# convert percentage to fraction
def convert_neutron(x):
    if x >= 1:
        return x / 100
    else:
        return x

beetaloo['NPHI'] = beetaloo['NPHI'].apply(convert_neutron)

In [8]:
# label encoding for well id
well_encoder = LabelEncoder()
beetaloo['WELL'] = well_encoder.fit_transform(beetaloo['WELL_ID'])

# label encoding for stratigraphy
strat_encoder = LabelEncoder()
beetaloo['STRAT_ENCODED'] = strat_encoder.fit_transform(beetaloo['STRAT'])

In [9]:
# columns
beetaloo.columns

Index(['WELL_ID', 'Y', 'X', 'DEPTH', 'RHOB', 'GR', 'DT', 'RES', 'SP', 'NPHI',
       'LITHO', 'STRAT', 'RES_10', 'WELL', 'STRAT_ENCODED'],
      dtype='object')

In [10]:
# feature selection
selected_features = ['WELL', 'X', 'Y', 'DEPTH', 
                     'RHOB', 'GR', 'DT', 'RES_10', 'SP', 'NPHI',
                     'STRAT_ENCODED']

## 2. Data Splitting

In [11]:
# copy of the dataset for modeling
data_ml = beetaloo[selected_features].copy()

In [12]:
# well test selection
test_wells = well_encoder.transform(['Tarlee_2', 'Beetaloo_W1', 'Friendship_1', 'Burdo_1', 'McManus_1', 'Amungee_NW1'])

# mask for test well
test_mask = data_ml['WELL'].isin(test_wells)

# column to identify train and test wells
data_ml['train_test'] = 'Train'
data_ml.loc[test_mask, 'train_test'] = 'Test'

# fraction of data
train_fraction = data_ml[data_ml['train_test'] == 'Train'].shape[0] / data_ml.shape[0]
test_fraction = data_ml[data_ml['train_test'] == 'Test'].shape[0] / data_ml.shape[0]
print(f"Fraction of data in train set: {train_fraction:.2f}")
print(f"Fraction of data in test set: {test_fraction:.2f}")
print(f"Total number of samples in dataset: {data_ml.shape[0]}")

Fraction of data in train set: 0.83
Fraction of data in test set: 0.17
Total number of samples in dataset: 352143


In [13]:
# create train and test sets
train = data_ml[~test_mask].copy()
test = data_ml[test_mask].copy()

## 3. Model Training

### 3.1. Input Values

In [14]:
# copy of train and test sets
X_train = train.copy()
X_test = test.copy()

In [15]:
# features to impute 
features_mice = ['RHOB', 'GR', 'DT', 'RES_10', 'SP', 'NPHI', 'STRAT_ENCODED']
imputed_cols = ['RHOB', 'GR', 'DT', 'RES_10', 'SP', 'NPHI']

In [16]:
# list with all combinations of well-logs for each well
unique_wells = X_train['WELL'].unique()
combinations = []
for well in unique_wells:
    for feature in imputed_cols:
        # only for well-logs that are not completely NaN
        if not X_train.loc[X_train['WELL'] == well, feature].isna().all():
            combinations.append((well, feature))
            
print('Number of Combinations:', len(combinations))

Number of Combinations: 119


In [17]:
# function to impute NaN values using iterative imputer (MICE)
def impute(train_data, cols_imp, model):
    mice = IterativeImputer(estimator=model, initial_strategy='mean' , random_state=17, max_iter=20, tol=0.01)
    mice.fit(train_data[cols_imp])
    imputed_train = mice.transform(train_data[cols_imp])
    return imputed_train, mice

### 3.2. Model Training with the best Hyperparameters

In [18]:
def training_model(X_train, model, param_grid, well_logs, cols_imp, n_splits, n_jobs):
    """
    Training Model with MICE
    
    Parameters
    ----------------------------------------------------------------------------------
        X_train: (pd.DataFrame) 
            Training data
        
        model: (model object for imputation)
            Model (e.g., KNeighborsRegressor, BayesianRidge, RF, XGBoost)
            
        param_grid: (dict)
            Dictionary of hyperparameters for the model
            
        well_logs: (list)
            List of well logs to evaluate
            
        cols_imp: (list)
            List of columns to impute
            
        n_splits: (int)
            Number of cross-validation splits
            
        n_jobs: (int)
            Number of jobs to run in parallel during imputation
    
    Returns
    ----------------------------------------------------------------------------------
        scaler: (object)
            MinMaxScaler object fitted on the imputed training data
            
        imp_model: (object)
            Trained imputation model
    """
    # copy of the data to work with
    data_train = X_train.copy()

    # scale the data for training
    scaler = MinMaxScaler()
    scaler.fit(data_train[cols_imp])
    X_training_scaled = scaler.transform(data_train[cols_imp])
    X_training_scaled_df = pd.DataFrame(X_training_scaled, 
                                        columns=cols_imp,
                                        index=X_train.index)

    # impute NaN values using iterative imputer
    if model == KNeighborsRegressor:
        X_training_imp, imp_model = impute(train_data= X_training_scaled_df,  
                                           cols_imp=cols_imp,
                                           model=model(**param_grid, n_jobs=n_jobs))

    elif model == BayesianRidge:
        X_training_imp, imp_model = impute(train_data=X_training_scaled_df,
                                           cols_imp=cols_imp,
                                           model=model(**param_grid))

    else:
        X_training_imp, imp_model = impute(train_data= X_training_scaled_df,  
                                           cols_imp=cols_imp,
                                           model=model(**param_grid, random_state=17, n_jobs=n_jobs))

    return scaler, imp_model

### KNeighborsRegressor

In [19]:
param_grid_knr = {}

In [20]:
scaler_knr, imp_model_knr = training_model(X_train=X_train, 
                                           model=KNeighborsRegressor, 
                                           param_grid=param_grid_knr,
                                           well_logs=imputed_cols,
                                           cols_imp=features_mice, 
                                           n_splits=5,
                                           n_jobs=-1)

C:\Users\lcbae\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


### BayesianRidge

In [21]:
param_grid_br = {}

In [22]:
scaler_br, imp_model_br = training_model(X_train=X_train, 
                                         model=BayesianRidge, 
                                         param_grid=param_grid_br,
                                         well_logs=imputed_cols,
                                         cols_imp=features_mice, 
                                         n_splits=5,
                                         n_jobs=-1)

### RandomForestRegressor

In [23]:
param_grid_rf = {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [24]:
scaler_rf, imp_model_rf = training_model(X_train=X_train, 
                                         model=RandomForestRegressor, 
                                         param_grid=param_grid_rf,
                                         well_logs=imputed_cols,
                                         cols_imp=features_mice, 
                                         n_splits=5,
                                         n_jobs=-1)

C:\Users\lcbae\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


### XGBRegressor

In [25]:
param_grid_xgb = {'max_depth': 6, 'reg_lambda': 2}

In [26]:
scaler_xgb, imp_model_xgb = training_model(X_train=X_train, 
                                           model=XGBRegressor, 
                                           param_grid=param_grid_xgb,
                                           well_logs=imputed_cols,
                                           cols_imp=features_mice, 
                                           n_splits=5,
                                           n_jobs=-1)

C:\Users\lcbae\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


## 4. Testing Imputation

In [27]:
def testing_model(X_test, scaler, imp_model, well_logs, cols_imp, n_splits):
    """
    Impute Test Data using Train Model
    
    Parameters
    ------------------------------------------------------------------------
        X_test: (pd.DataFrame) 
            Test data
        
        scaler: (scaler object)
            Scaler object fitted on the training data
            
        imp_model: (imputation model object)
            Imputation model trained on the training data
            
        well_logs: (list)
            List of well logs to evaluate
            
        cols_imp: (list)
            List of columns to impute
            
        n_splits: (int)
            Number of cross-validation splits
    
    Returns
    ------------------------------------------------------------------------
        combined_df: (pd.DataFrame)
            Combined results dataframe containing original scaled values, 
            scaled imputed values, and imputed values for each well log
    """
    # scale the original test data
    X_test_original_scale = scaler.transform(X_test[cols_imp])        
    X_test_original_scale_df = pd.DataFrame(X_test_original_scale,
                                      columns=cols_imp,
                                      index=X_test.index)
    X_test_original_scale_df['WELL'] = X_test['WELL']
    
    unique_wells = X_test['WELL'].unique()
    combinations = []
    for well in unique_wells:
        for feature in imputed_cols:
            # only for well-logs that are not completely NaN
            if not X_test.loc[X_test['WELL'] == well, feature].isna().all():
                combinations.append((well, feature))
    
    kf = KFold(n_splits=n_splits, random_state=17, shuffle=True)
    
    X_test_sc_imp_result = X_test.copy()
    X_test_imp_result = X_test.copy()
     
    for i, (test_index, val_index) in enumerate(kf.split(combinations)):

        # test and validations sets
        test_combinations = [combinations[i] for i in test_index]
        validation_combinations = [combinations[i] for i in val_index]

        # copy of the data to work with
        data_test = X_test.copy()
        
        # set values to NaN in the data to impute using the validation combinations
        for well_id, feature_name in validation_combinations:
            data_test.loc[data_test['WELL']==well_id, feature_name] = np.nan


        # scale the test with NaN using the scaler object fitted on the training data
        X_test_scaled = scaler.transform(data_test[cols_imp])
        X_test_scaled_df = pd.DataFrame(X_test_scaled, 
                                        columns=cols_imp,
                                        index=X_test.index)

        # impute NaN values in the test data using the imp_model trained on the training data
        X_test_imp = imp_model.transform(X_test_scaled_df[cols_imp])
        X_test_imp_scaled = pd.DataFrame(X_test_imp, columns=cols_imp, index=X_test.index)

        
        # inverse transform the imputed values
        X_test_imp_unscaled = scaler.inverse_transform(X_test_imp_scaled)
        X_test_imp_df = pd.DataFrame(X_test_imp_unscaled, 
                                     columns=cols_imp, 
                                     index=X_test.index)
        
        
        # store results of the scaled imputation using validation combinations
        for well_id, feature_name in validation_combinations:
            scaled_imputed_result = X_test_imp_scaled.loc[X_test['WELL']==well_id, feature_name]
            X_test_sc_imp_result.loc[X_test['WELL']==well_id, feature_name] = scaled_imputed_result
        
        # store results of the imputation using validation combinations
        for well_id, feature_name in validation_combinations:
            imputed_result = X_test_imp_df.loc[X_test['WELL']==well_id, feature_name]
            X_test_imp_result.loc[X_test['WELL']==well_id, feature_name] = imputed_result
                
    # rename columns
    X_test_original_scale_df.rename(columns=lambda x: x + '_SCALED', inplace=True)
    X_test_sc_imp_result.rename(columns=lambda x: x + '_IMP_SCALED', inplace=True)
    X_test_imp_result.rename(columns=lambda x: x + '_IMP', inplace=True)
    
    # combine dataframes
    combined_df = pd.concat([X_test_original_scale_df[[column + '_SCALED' for column in well_logs]],
                             X_test_sc_imp_result[[column + '_IMP_SCALED' for column in well_logs]],
                             X_test_imp_result[[column + '_IMP' for column in well_logs]]
                            ], axis=1)

    return  combined_df

### KNeighborsRegressor

In [28]:
test_result_knr = testing_model(X_test=X_test, 
                                scaler=scaler_knr, 
                                imp_model=imp_model_knr,
                                well_logs=imputed_cols,
                                cols_imp=features_mice,
                                n_splits=5
                               )

In [29]:
# save results to csv file
test_result_knr.to_csv('test_result_knr.csv', index=False)

### BayesianRidge

In [30]:
test_result_br = testing_model(X_test=X_test, 
                               scaler=scaler_br, 
                               imp_model=imp_model_br,
                               well_logs=imputed_cols,
                               cols_imp=features_mice,
                               n_splits=5
                              )

In [31]:
# save results to csv file
test_result_br.to_csv('test_result_br.csv', index=False)

### RandomForestRegressor

In [32]:
test_result_rf = testing_model(X_test=X_test, 
                               scaler=scaler_rf, 
                               imp_model=imp_model_rf,
                               well_logs=imputed_cols,
                               cols_imp=features_mice,
                               n_splits=5
                              )

In [33]:
# save results to csv file
test_result_rf.to_csv('test_result_rf.csv', index=False)

### XGBRegressor

In [34]:
test_result_xgb = testing_model(X_test=X_test, 
                                scaler=scaler_xgb, 
                                imp_model=imp_model_xgb,
                                well_logs=imputed_cols,
                                cols_imp=features_mice,
                                n_splits=5
                               )

In [35]:
# save results to csv file
test_result_xgb.to_csv('test_result_xgb.csv', index=False)